## What is Model Validation

* 모델 성능(model quality)의 대부분은 **예측 정확도**와 관련이 있음
* 많은 사람들이 예측 정확도를 측정할 때, 학습에 사용한 데이터로 예측하고 결과를 비교함 -> **mistake**
<br>

* Mean Absolute Error(MAE): 모델 성능 평가를 위한 여러 측정법 중 하나(간단하게 **error**라 부름)
    * `error = actual - predicted`
    * 위 수식이 각각의 데이터에 대해 수행됨
    * 예를 들어, 실제 집 값이 \$150,000이고, 예측한 값이 \$100,000일 경우, error는 \$50,000이 됨
    * 각 데이터에 대한 error들을 평균내면, 그 값이 모델의 성능이 됨
    * 따라서, X에 대한 우리의 예측 값은 평균(모델의 성능)값만큼 적거나 클 수 있음

In [3]:
import pandas as pd

## Load data ##
melbourne_file_path = '../input/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

## Filter rows with missing price values ##
filtered_melbourne_data = melbourne_data.dropna(axis=0)

## Choose target and features ##
y = filtered_melbourne_data.Price
melbourne_features = ["Rooms", "Bathroom", "Landsize", "BuildingArea", "YearBuilt", "Lattitude", "Longtitude"]
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor

## Define model ##
melbourne_model = DecisionTreeRegressor()

## Fit model ##
melbourne_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [5]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

<br><br>

## The Problem with "In-Sample" Scores

* 위에서 구한 error는 "in-sample" 스코어. 모델을 모델링하고 평가하는데 단일 샘플(학습/평가 데이터가 구분되지 않음)만 사용함
    * In Sample Error: The error rate you get on the same data set you used to build your predictor.
* 학습 데이터로 학습된 모델은 학습 데이터에선 정확한 예측을 하지만, 새로운 데이터에선 예측을 잘 하지 못함
* 모델의 성능 평가는 모델이 이전에 보지못한(모델링에 사용하지 않은) 데이터로 이루어짐
* 모델의 성능 평가에 사용되는 데이터를 **validation data**라고 함

## Coding it

* train_test_split(arrays, test_size=0.25, train_size=1-test_size, random_state, shuffle=True, stratify): scikit-learn의 메소드. 데이터셋을 두 그룹으로 나눠줌
    * arrays: 전체 데이터셋
    * test_size: 전체 데이터셋에서 validation data로 사용할 데이터의 비율
    * train_size: test_size의 나머지
    * random_state: random을 위한 seed
    * shuffle: 데이터 셔플 여부
    * stratify: 데이터 값의 비율 유지 여부(stratify=y)
        * 예를 들어, **y가 25%의 0, 75%의 1**로 이루어져있을 때 stratify=y로 설정하면 나눠진 데이터 내의 0과 1 비율도 그대로(0:25%, 1:75%) 유지됨

* 학습 데이터(training data)는 모델을 fit하는데 사용하고, 평가 데이터(validation data)는 error(MAE)를 계산하는 데 사용

In [7]:
from sklearn.model_selection import train_test_split

"""
split data into training and validation data, for both features and target
The split is based on a random number generator. Supplying a numeric value to
the random_state argument guarantees we get the same split every tume we
run this script.
"""
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

## Define model ##
melbourne_model = DecisionTreeRegressor()

## Fit model ##
melbourne_model.fit(train_X, train_y)

## get predicted prices on validation data ##
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

262095.61200774694


* in-sample 데이터에서 error는 약 500이었는데, out-of-sample 데이터에서 error는 약 260,000